# 31. Auditoria: Agente de Análise de Diário Oficial (PDF)

Este notebook demonstra como criar um "Agente Auditor" que:
1.  Lê um arquivo PDF de Diário Oficial (simulado).
2.  Extrai estruturadamente todas as compras e contratações usando **Google Gemini** e **Pydantic**.
3.  Analisa os dados extraídos em busca de anomalias ou pontos de atenção.
4.  Gera um Relatório Executivo automático.

**Fluxo:** PDF -> Texto -> Extração Estruturada (LLM) -> Análise de Regras -> Relatório Final.

In [ ]:
### INJECTION START ###
import os
from dotenv import load_dotenv
# Carregar variáveis de ambiente locais se existirem
load_dotenv()
### INJECTION END ###

# Instalação das dependências
!pip install -qU langchain langchain-google-genai pypdf fpdf

## 1. Gerando um Diário Oficial de Exemplo (PDF)

Para tornar este exemplo reprodutível, vamos criar um PDF fictício contendo algumas publicações de compras, algumas normais e outras suspeitas.

In [ ]:
from fpdf import FPDF
import os

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'DIÁRIO OFICIAL DO MUNICÍPIO - EXEMPLO', 0, 1, 'C')
        self.ln(10)

def create_dummy_pdf(filename="diario_exemplo.pdf"):
    pdf = PDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    
    texto = """
    ATOS DO PODER EXECUTIVO
    
    DECRETO Nº 123/2024 - Nomeia servidor João da Silva para cargo de Assessor.
    
    EXTRATO DE CONTRATO Nº 001/2024
    Processo Administrativo 50/2023.
    Contratante: Secretaria de Educação.
    Contratada: PAPELARIA EXEMPLAR LTDA.
    Objeto: Aquisição de material de escritório (lápis, caneta, papel A4) para as escolas municipais.
    Valor: R$ 15.000,00 (Quinze mil reais).
    Vigência: 12 meses.
    
    AVISO DE LICITAÇÃO DESERTA
    Pregão Eletrônico 05/2024. Objeto: Reforma da Praça Central. Nenhuma empresa interessada.
    
    EXTRATO DE CONTRATO Nº 002/2024
    Processo: Dispensa de Licitação 02/2024.
    Contratante: Fundo Municipal de Saúde.
    Contratada: SUPER TECH INOVAÇÕES EIRELI.
    Objeto: Fornecimento de 5 notebooks de alto desempenho para o gabinete do Secretário.
    Valor: R$ 85.000,00 (Oitenta e cinco mil reais).
    Nota: Valor unitário de R$ 17.000,00.
    
    EXTRATO DE TERMO ADITIVO
    Contrato Original 10/2022. Empresa: Construtora Pedra Fundamental.
    Objeto: Prorrogação de prazo por mais 6 meses para conclusão da obra da creche.
    Valor Adicional: R$ 0,00.
    
    EXTRATO DE COMPRA DIRETA
    Autorização: Secretaria de Obras.
    Fornecedor: Irmãos Silva Materiais de Construção.
    Objeto: Compra de 50 sacos de cimento emergencial.
    Valor: R$ 2.500,00.
    """
    
    # Escrevendo linha a linha para evitar problemas de quebra
    for line in texto.split('\n'):
        pdf.multi_cell(0, 8, line.strip())
        
    pdf.output(filename)
    print(f"Arquivo '{filename}' gerado com sucesso.")

create_dummy_pdf()

## 2. Carregando e Processando o PDF

Uso do `PyPDFLoader` para obter o texto bruto.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("diario_exemplo.pdf")
pages = loader.load()
full_text = "\n".join([p.page_content for p in pages])

print(f"Texto carregado ({len(full_text)} caracteres).")
# print(full_text) # Opcional: imprimir texto para conferir

## 3. Definindo o Schema de Extração (Pydantic)

Definimos exatamente quais dados queremos capturar de cada compra.

In [ ]:
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class Compra(BaseModel):
    """Representa uma compra ou contrato extraído do diário oficial."""
    numero_contrato: Optional[str] = Field(None, description="Número do contrato ou processo")
    contratante: str = Field(..., description="Órgão ou secretaria que está comprando")
    contratada: str = Field(..., description="Nome da empresa ou pessoa contratada")
    objeto: str = Field(..., description="Descrição do bem ou serviço adquirido")
    valor: float = Field(..., description="Valor total da compra em Reais (converter para float)")
    modalidade: Optional[str] = Field(None, description="Modalidade (Licitação, Dispensa, Pregão, etc)")

class DiarioOficial(BaseModel):
    """Lista de todas as compras encontradas no texto."""
    compras: List[Compra] = Field(description="Lista de compras e contratos identificados")

## 4. Agente de Extração (Gemini)

Usamos `with_structured_output` para obrigar o LLM a retornar JSON conforme nosso schema.

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Configuração da API
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "SUA_CHAVE_AQUI" # No Colab use userdata
    try:
        from google.colab import userdata
        os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    except:
        pass

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

structured_llm = llm.with_structured_output(DiarioOficial)

prompt = ChatPromptTemplate.from_template(
    """Você é um assistente de IA especializado em ler Diários Oficiais.
    Extraia TODAS as informações sobre compras, contratos e licitações do texto abaixo.
    Se não encontrar o valor exato, tente estimar ou extrair o número disponível.
    Ignore nomeações de pessoal ou decretos que não envolvam gastos com fornecedores.
    
    TEXTO DO DIÁRIO:
    {text}
    """
)

chain_extract = prompt | structured_llm

In [ ]:
print("⏳ Extraindo dados com IA...")
resultado = chain_extract.invoke({"text": full_text})

print(f"✅ Extração completa! Encontradas {len(resultado.compras)} compras.")
for i, compra in enumerate(resultado.compras):
    print(f"{i+1}. {compra.contratada} | R$ {compra.valor:,.2f} | {compra.objeto[:50]}...")

## 5. Agente de Auditoria (Análise)

Agora vamos criar uma função que avalia as compras extraídas e busca "red flags" (bandeiras vermelhas).

In [ ]:
def analisar_compras(dados: DiarioOficial) -> str:
    analise_prompt = ChatPromptTemplate.from_template(
        """Você é um Auditor Geral. Analise a lista de compras extraídas do Diário Oficial abaixo.
        Identifique possíveis irregularidades, preços abusivos ou descrições vagas. 
        Dê atenção especial a compras de alto valor ou itens de luxo/desnecessários.
        
        DADOS DAS COMPRAS:
        {dados_json}
        
        Gere um RELATÓRIO EXECUTIVO em Markdown contendo:
        1. Resumo dos gastos (Total, Maior Compra).
        2. Pontos de Atenção (Red Flags) - com justificativa.
        3. Recomendações de Fiscalização.
        """
    )
    
    # Convertendo os dados Pydantic para JSON string para o prompt
    dados_json = dados.json()
    
    chain_analise = analise_prompt | llm
    response = chain_analise.invoke({"dados_json": dados_json})
    return response.content

print("📊 Gerando Relatório de Auditoria...")
relatorio = analisar_compras(resultado)
from IPython.display import Markdown
display(Markdown(relatorio))

### Conclusão

Criamos um pipeline completo 'End-to-End':
1.  O PDF entra no sistema.
2.  O Gemini estrutura os dados não estruturados.
3.  O Gemini (em outra chamada) atua como auditor e gera inteligência sobre os dados.